In [ ]:
# import files from parent directory
import sys
sys.path.append("..")

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

from utils.network_loader import setup_dyetracing_graph, read_communities, load_nx_graph
from utils.dyetracing_classes import run_simulation, cluster_and_evaluate
from utils.node_selection import find_low_betweenness_nodes

In [ ]:
#import network and community file
network_file = '../benchmarks/LF_created_networks/500_node/30/1_network_N500_k50_maxk75_mu30.dat'
community_file = '../benchmarks/LF_created_networks/500_node/30/1_community_N500_k50_maxk75_mu30.dat'

#convert network and community file to graph and group membership
node_list, edge_list = setup_dyetracing_graph(network_file)  
ground_truth = np.array(list(read_communities(community_file).values()))
print("number of communities: ", len(np.unique(ground_truth)))

#create networkx graph for visualization and source node selection
G_directed = load_nx_graph(node_list, network_file)

In [ ]:
#select source nodes
source_nodes = find_low_betweenness_nodes(G_directed, node_list, 100)

#set params and run simulation
params = {"num_steps": 50, "absorption_rate": 1, "absorption_limit": 0.4, "decay_rate": 0}
flow_profile = run_simulation(node_list, edge_list, source_nodes, params)

In [ ]:
# get simulation results
results = cluster_and_evaluate(flow_profile, range(3,35), plot=True, true_membership=ground_truth)

#get community labels
labels = results["optimal_labels"]
nmi = results["nmi"]


In [ ]:
#plot graph with colored communities
pos = nx.spring_layout(G_directed, seed=1, k=1.5)
nx.draw_networkx_nodes(G_directed, pos, node_size=100, node_color=labels, cmap="tab20", edgecolors='grey', linewidths=1.2)
# nx.draw_networkx_edges(G_directed, pos, alpha=0.2, width=1.3, edge_color='grey') #edges not shown for better visibility

plt.title(f"Directed Flow Communities on LFR network\nNMI: {nmi:.2f}")
plt.axis('off')
plt.show()